In [1]:
import numpy
from sklearn.feature_extraction import DictVectorizer 
import h2o 
from h2o.estimators.random_forest import H2ORandomForestEstimator 
from h2o.estimators.kmeans import H2OKMeansEstimator 
import pandas 
from function import * 
pandas.options.display.max_rows=1000 

In [2]:
h2o.init()

Connecting to H2O server at http://localhost:54321... successful!


H2O cluster uptime:,16 hours 27 mins
H2O cluster version:,3.10.0.3
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_I_1kgl2b
H2O cluster total nodes:,1
H2O cluster free memory:,3.161 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster is healthy:,True
H2O cluster is locked:,True
H2O connection url:,http://localhost:54321


In [3]:
collection = mongo_collection() 
records = collection.find({'$or': [{'type': 'apartments'}, {'type': 'hotel'}]})

In [4]:
train_records, valid_records = split_records(records, 0.8)

In [5]:
train_features, train_y = hotel_records(train_records) 
valid_features, valid_y = hotel_records(valid_records) 

In [6]:
train_vectorizer = DictVectorizer() 
X_train = train_vectorizer.fit_transform(train_features)

In [7]:
train = pandas.DataFrame(X_train.toarray(), columns=normalize_features(train_vectorizer.get_feature_names())) 
train['type'] = train_y  
X_valid = train_vectorizer.transform(valid_features)
valid = pandas.DataFrame(X_valid.toarray(), columns=normalize_features(train_vectorizer.get_feature_names())) 
valid['type'] = valid_y  

In [8]:
train_water = h2o.H2OFrame.from_python(train.to_dict('list')) 
valid_water = h2o.H2OFrame.from_python(valid.to_dict('list'))

In [9]:
pandas.options.display.max_rows=1000 

In [10]:
rf = H2ORandomForestEstimator(ntrees=100, max_depth=25, min_rows=5, col_sample_rate_change_per_level=1.65)

In [11]:
xc = train_water.columns.copy() 
xc.remove('type')  

In [12]:
rf.train(training_frame=train_water, validation_frame=valid_water, x=xc, y='type')

In [13]:
rf

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1472160914002_670

Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,100.0,100.0,2524075.0,25.0,25.0,25.0,1830.0,2206.0,1983.06




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.048781407526539027
RMSE: 0.22086513424834403
LogLoss: 0.17526144198806184
Mean Per-Class Error: 0.06201936914635908
AUC: 0.9815371895212671
Gini: 0.9630743790425342

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.49330061515368745: 


,apartments,hotel,Error,Rate
apartments,34658.0,2339.0,0.0632,(2339.0/36997.0)
hotel,3027.0,45698.0,0.0621,(3027.0/48725.0)
Total,37685.0,48037.0,0.0626,(5366.0/85722.0)



Maximum Metrics: Maximum metrics at their respective thresholds



...



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2016-08-26 07:09:03,0.005 sec,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
,2016-08-26 07:09:05,1.980 sec,1.0,0.2696977,1.1737378,0.9480174,1.7357926,0.0904543,0.2649360,1.0932655,0.9510496,1.7083693,0.0869385
,2016-08-26 07:09:07,3.633 sec,2.0,0.2639267,1.0204681,0.9523830,1.7558548,0.0881054,0.2424095,0.5249553,0.9682496,1.7380241,0.0740245
,2016-08-26 07:09:11,8.373 sec,5.0,0.2511186,0.6983196,0.9613516,1.7593022,0.0796617,0.2296954,0.2641704,0.9764779,1.7538304,0.0663684
,2016-08-26 07:09:16,13.346 sec,8.0,0.2427080,0.4964133,0.9680188,1.7593022,0.0747383,0.2265636,0.2169327,0.9785794,1.7576200,0.0654921
,2016-08-26 07:09:23,19.931 sec,12.0,0.2342447,0.3344698,0.9737569,1.7593022,0.0702157,0.2240250,0.2010125,0.9796752,1.7576847,0.0637856
,2016-08-26 07:09:30,26.661 sec,16.0,0.2295402,0.2586358,0.9769290,1.7593022,0.0671391,0.2216015,0.1841073,0.9808275,1.7601932,0.0627710
,2016-08-26 07:09:38,34.820 sec,21.0,0.2269272,0.2231806,0.9783954,1.7593022,0.0653313,0.2208316,0.1788136,0.9812751,1.7603017,0.0620791
,2016-08-26 07:09:46,43.103 sec,26.0,0.2250700,0.2074747,0.9793272,1.7593022,0.0646742,0.2203636,0.1782862,0.9813422,1.7597278,0.0619408
,2016-08-26 07:09:56,52.803 sec,32.0,0.2239206,0.1935256,0.9800445,1.7593022,0.0642542,0.2200446,0.1750153,0.9815094,1.7608063,0.0620330



Variable Importances: 


variable,relative_importance,scaled_importance,percentage
_24_hour_front_desk,128999.4531250,1.0,0.0998468
_Stovetop,92087.9609375,0.7138632,0.0712769
_Kitchenware,86546.7578125,0.6709079,0.0669880
_Meeting_banquet_facilities,79047.2890625,0.6127723,0.0611833
_Kitchen,58415.7968750,0.4528376,0.0452144
---,---,---,---
_Game_console___PS4,0.0,0.0,0.0
_Game_console___PS3,0.0,0.0,0.0
_Public_Bath,0.0,0.0,0.0
_Game_console___Xbox_360,0.0,0.0,0.0



See the whole table with table.as_data_frame()


In [14]:
rf.model_performance()
    



ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.048781407526539027
RMSE: 0.22086513424834403
LogLoss: 0.17526144198806184
Mean Per-Class Error: 0.06201936914635908
AUC: 0.9815371895212671
Gini: 0.9630743790425342

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.49330061515368745: 


,apartments,hotel,Error,Rate
apartments,34658.0,2339.0,0.0632,(2339.0/36997.0)
hotel,3027.0,45698.0,0.0621,(3027.0/48725.0)
Total,37685.0,48037.0,0.0626,(5366.0/85722.0)



Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4933006,0.9445443,195.0
max f2,0.2394356,0.9539671,270.0
max f0point5,0.7274129,0.9573584,131.0
max accuracy,0.4969978,0.9374373,194.0
max precision,0.9996951,0.9985781,1.0
max recall,0.0001620,1.0,399.0
max specificity,0.9999472,0.9996486,0.0
max absolute_mcc,0.4969978,0.8729355,194.0
max min_per_class_accuracy,0.4969978,0.9372807,194.0
max mean_per_class_accuracy,0.5608778,0.9379806,176.0



Gains/Lift Table: Avg response rate: 56.84 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0139637,1.0,1.7593022,1.7593022,1.0,1.0,0.0245664,0.0245664,75.9302206,75.9302206
,2,0.0200882,0.9999909,1.7593022,1.7593022,1.0,1.0,0.0107748,0.0353412,75.9302206,75.9302206
,3,0.0310539,0.9999736,1.7593022,1.7593022,1.0,1.0,0.0192919,0.0546331,75.9302206,75.9302206
,4,0.0400014,0.9999519,1.7593022,1.7593022,1.0,1.0,0.0157414,0.0703746,75.9302206,75.9302206
,5,0.0500338,0.9999212,1.7593022,1.7593022,1.0,1.0,0.0176501,0.0880246,75.9302206,75.9302206
,6,0.1000093,0.9996353,1.7593022,1.7593022,1.0,1.0,0.0879220,0.1759466,75.9302206,75.9302206
,7,0.1504165,0.9980000,1.7593022,1.7593022,1.0,1.0,0.0886814,0.2646280,75.9302206,75.9302206
,8,0.2000537,0.9955556,1.7593022,1.7593022,1.0,1.0,0.0873268,0.3519548,75.9302206,75.9302206
,9,0.3000047,0.9850109,1.7593022,1.7593022,1.0,1.0,0.1758440,0.5277989,75.9302206,75.9302206
,10,0.4000023,0.9576347,1.7593022,1.7593022,1.0,1.0,0.1759261,0.7037250,75.9302206,75.9302206


In [15]:
rf.varimp(use_pandas=True)

,variable,relative_importance,scaled_importance,percentage
0,_24_hour_front_desk,128999.453125,1.000000e+00,9.984679e-02
1,_Stovetop,92087.960938,7.138632e-01,7.127695e-02
2,_Kitchenware,86546.757812,6.709079e-01,6.698800e-02
3,_Meeting_banquet_facilities,79047.289062,6.127723e-01,6.118334e-02
4,_Kitchen,58415.796875,4.528376e-01,4.521438e-02
5,_Bar,52128.644531,4.040997e-01,4.034806e-02
6,_Oven,49609.542969,3.845717e-01,3.839825e-02
7,_Family_rooms,44062.660156,3.415725e-01,3.410491e-02
8,_Telephone,42844.535156,3.321296e-01,3.316207e-02
9,_Fax_photocopying,42472.718750,3.292473e-01,3.287428e-02


In [16]:
rf.show()

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1472160914002_670

Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,100.0,100.0,2524075.0,25.0,25.0,25.0,1830.0,2206.0,1983.06




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.048781407526539027
RMSE: 0.22086513424834403
LogLoss: 0.17526144198806184
Mean Per-Class Error: 0.06201936914635908
AUC: 0.9815371895212671
Gini: 0.9630743790425342

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.49330061515368745: 


,apartments,hotel,Error,Rate
apartments,34658.0,2339.0,0.0632,(2339.0/36997.0)
hotel,3027.0,45698.0,0.0621,(3027.0/48725.0)
Total,37685.0,48037.0,0.0626,(5366.0/85722.0)



Maximum Metrics: Maximum metrics at their respective thresholds



...



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2016-08-26 07:09:03,0.005 sec,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
,2016-08-26 07:09:05,1.980 sec,1.0,0.2696977,1.1737378,0.9480174,1.7357926,0.0904543,0.2649360,1.0932655,0.9510496,1.7083693,0.0869385
,2016-08-26 07:09:07,3.633 sec,2.0,0.2639267,1.0204681,0.9523830,1.7558548,0.0881054,0.2424095,0.5249553,0.9682496,1.7380241,0.0740245
,2016-08-26 07:09:11,8.373 sec,5.0,0.2511186,0.6983196,0.9613516,1.7593022,0.0796617,0.2296954,0.2641704,0.9764779,1.7538304,0.0663684
,2016-08-26 07:09:16,13.346 sec,8.0,0.2427080,0.4964133,0.9680188,1.7593022,0.0747383,0.2265636,0.2169327,0.9785794,1.7576200,0.0654921
,2016-08-26 07:09:23,19.931 sec,12.0,0.2342447,0.3344698,0.9737569,1.7593022,0.0702157,0.2240250,0.2010125,0.9796752,1.7576847,0.0637856
,2016-08-26 07:09:30,26.661 sec,16.0,0.2295402,0.2586358,0.9769290,1.7593022,0.0671391,0.2216015,0.1841073,0.9808275,1.7601932,0.0627710
,2016-08-26 07:09:38,34.820 sec,21.0,0.2269272,0.2231806,0.9783954,1.7593022,0.0653313,0.2208316,0.1788136,0.9812751,1.7603017,0.0620791
,2016-08-26 07:09:46,43.103 sec,26.0,0.2250700,0.2074747,0.9793272,1.7593022,0.0646742,0.2203636,0.1782862,0.9813422,1.7597278,0.0619408
,2016-08-26 07:09:56,52.803 sec,32.0,0.2239206,0.1935256,0.9800445,1.7593022,0.0642542,0.2200446,0.1750153,0.9815094,1.7608063,0.0620330



Variable Importances: 


variable,relative_importance,scaled_importance,percentage
_24_hour_front_desk,128999.4531250,1.0,0.0998468
_Stovetop,92087.9609375,0.7138632,0.0712769
_Kitchenware,86546.7578125,0.6709079,0.0669880
_Meeting_banquet_facilities,79047.2890625,0.6127723,0.0611833
_Kitchen,58415.7968750,0.4528376,0.0452144
---,---,---,---
_Game_console___PS4,0.0,0.0,0.0
_Game_console___PS3,0.0,0.0,0.0
_Public_Bath,0.0,0.0,0.0
_Game_console___Xbox_360,0.0,0.0,0.0



See the whole table with table.as_data_frame()


In [17]:
rf.confusion_matrix(valid=True, metrics = ['f1', 'f2', 'absolute_mcc', 'accuracy', 'precision', 'f0point5', 'min_per_class_accuracy', 'mean_per_class_accuracy'])


Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4482696351661746: 


,apartments,hotel,Error,Rate
apartments,8729.0,662.0,0.0705,(662.0/9391.0)
hotel,669.0,11622.0,0.0544,(669.0/12291.0)
Total,9398.0,12284.0,0.0614,(1331.0/21682.0)



Confusion Matrix (Act/Pred) for max f2 @ threshold = 0.23469131687389955: 


,apartments,hotel,Error,Rate
apartments,7979.0,1412.0,0.1504,(1412.0/9391.0)
hotel,331.0,11960.0,0.0269,(331.0/12291.0)
Total,8310.0,13372.0,0.0804,(1743.0/21682.0)



Confusion Matrix (Act/Pred) for max absolute_mcc @ threshold = 0.4838356823012388: 


...


Confusion Matrix (Act/Pred) for max min_per_class_accuracy @ threshold = 0.49049135719115533: 


,apartments,hotel,Error,Rate
apartments,8811.0,580.0,0.0618,(580.0/9391.0)
hotel,761.0,11530.0,0.0619,(761.0/12291.0)
Total,9572.0,12110.0,0.0618,(1341.0/21682.0)



Confusion Matrix (Act/Pred) for max mean_per_class_accuracy @ threshold = 0.5182547735545086: 


,apartments,hotel,Error,Rate
apartments,8869.0,522.0,0.0556,(522.0/9391.0)
hotel,819.0,11472.0,0.0666,(819.0/12291.0)
Total,9688.0,11994.0,0.0618,(1341.0/21682.0)


[, , , , , , , ]

In [18]:
rf.confusion_matrix(valid=True, metrics = ['min_per_class_accuracy', 'mean_per_class_accuracy'])


Confusion Matrix (Act/Pred) for max min_per_class_accuracy @ threshold = 0.49049135719115533: 


,apartments,hotel,Error,Rate
apartments,8811.0,580.0,0.0618,(580.0/9391.0)
hotel,761.0,11530.0,0.0619,(761.0/12291.0)
Total,9572.0,12110.0,0.0618,(1341.0/21682.0)



Confusion Matrix (Act/Pred) for max mean_per_class_accuracy @ threshold = 0.5182547735545086: 


,apartments,hotel,Error,Rate
apartments,8869.0,522.0,0.0556,(522.0/9391.0)
hotel,819.0,11472.0,0.0666,(819.0/12291.0)
Total,9688.0,11994.0,0.0618,(1341.0/21682.0)


[, ]

#K-means Model 

In [32]:
kmeans = H2OKMeansEstimator(k=2) 

In [40]:
kmeans.train(training_frame=train_water, x=xc)

In [41]:
kmeans 

Model Details
H2OKMeansEstimator :  K-means
Model Key:  KMeans_model_python_1472160914002_740

Model Summary: 


,number_of_rows,number_of_clusters,number_of_categorical_columns,number_of_iterations,within_cluster_sum_of_squares,total_sum_of_squares,between_cluster_sum_of_squares
,85722.0,2.0,0.0,30.0,22474128.1429083,23658997.0000000,1184868.8570917




ModelMetricsClustering: kmeans
** Reported on train data. **

MSE: NaN
RMSE: NaN
Total Within Cluster Sum of Square Error: 22474128.14290835
Total Sum of Square Error to Grand Mean: 23658997.0
Between Cluster Sum of Square Error: 1184868.8570916504

Centroid Statistics: 


,centroid,size,within_cluster_sum_of_squares
,1.0,28549.0,9736819.3726369
,2.0,57173.0,12737308.7702715



Scoring History: 


,timestamp,duration,iteration,avg_change_of_std_centroids,within_cluster_sum_of_squares
,2016-08-26 07:51:21,0.001 sec,0.0,nan,33505841.0288638
,2016-08-26 07:51:23,1.964 sec,1.0,354.7968502,23586002.6827574
,2016-08-26 07:51:23,2.234 sec,2.0,72.8310401,23460134.9268176
,2016-08-26 07:51:23,2.497 sec,3.0,21.1558418,23356231.9855523
,2016-08-26 07:51:23,2.774 sec,4.0,10.9266325,23230774.9719929
---,---,---,---,---,---
,2016-08-26 07:51:29,8.533 sec,25.0,0.0009812,22474160.1831196
,2016-08-26 07:51:29,8.815 sec,26.0,0.0002402,22474136.6315281
,2016-08-26 07:51:30,9.091 sec,27.0,0.0000663,22474129.9914802
,2016-08-26 07:51:30,9.378 sec,28.0,0.0000177,22474128.4165431



See the whole table with table.as_data_frame()


In [42]:
kmeans.centers() 

[[0.016007565939262374,
  0.025605100003502725,
  0.002311814774598145,
  0.0025219797541069966,
  0.14490875337139658,
  0.0014711548565623986,
  0.09415391081999358,
  0.00021016497950891732,
  0.0028021997267853108,
  0.0002451924760937346,
  0.029948509580020544,
  0.03054397702196229,
  0.006024729412588883,
  0.023748642684507274,
  0.007110581806718235,
  0.0009457424077901136,
  0.025324880030824157,
  0.0018564573189953997,
  0.1612665942765084,
  0.2931451189183506,
  0.017969105748012124,
  0.07989771970997195,
  0.001751374829240801,
  0.02669095239763325,
  0.01576237346316464,
  0.009982836526673445,
  0.0036778871414060357,
  0.19590878839889284,
  0.8217100423832719,
  0.0077410767452449885,
  0.005569371956986231,
  0.052436162387473964,
  0.003642859644821172,
  0.10697397457003803,
  0.5026445759921502,
  0.03958107114084585,
  0.03127955445024346,
  0.051735612455777236,
  0.04459000315247444,
  0.037584503835509986,
  0.003467722161897064,
  0.0028021997267855415,


In [43]:
kmeans.centers_std()

[[0.08984317065320259, -0.04486265683064153],
 [-0.06231943011749455, 0.03111883949459266],
 [-0.0546215495511399, 0.02727494828215377],
 [0.04862450516708894, -0.024280359575590223],
 [-0.27472040712811013, 0.13718001334721172],
 [0.03677364291356604, -0.018362701476910187],
 [0.11303701245099233, -0.05644436479568152],
 [0.016754914831997838, -0.008366467800162314],
 [-0.12406810302067503, 0.06195267474397295],
 [0.018097481975188817, -0.009036870776584641],
 [-0.13066783960566822, 0.06524821424277845],
 [-0.05572510899182043, 0.027826004173430386],
 [-0.03936512670578591, 0.019656743608408092],
 [-0.04739782956937924, 0.023667826358179333],
 [0.044048898140053666, -0.02199555722107376],
 [-0.134202710901434, 0.06701333135439963],
 [0.003292166709471149, -0.0016439240093873858],
 [-0.012209114626912388, 0.006096549306205462],
 [-0.31273058754498684, 0.15616017252587724],
 [0.5380045593091379, -0.2686494003063844],
 [-0.15624660398681778, 0.07802081921920434],
 [0.15868862880476978, -

In [44]:
centroids = pandas.DataFrame(kmeans.centers_std())

In [50]:
features = train_water.columns.copy() 
features.remove('type')

In [51]:
centroids['Variable'] = features 
centroids 

,0,1,Variable
0,0.089843,-0.044863,_Shared_Toilet
1,-0.062319,0.031119,_Private_beach_area
2,-0.054622,0.027275,_Pool_bar
3,0.048625,-0.024280,_Child_safety_socket_covers
4,-0.274720,0.137180,_Fitness_centre
5,0.036774,-0.018363,_Game_console___Nintendo_Wii
6,0.113037,-0.056444,_Beachfront
7,0.016755,-0.008366,_Raised_toilet
8,-0.124068,0.061953,_Evening_entertainment
9,0.018097,-0.009037,_Shower_chair


In [ ]:
centroids.describe()

In [47]:
len(xc)

276

False

In [49]:
len(features)

277